# Collision Avoidance - Live View

This notebook will be used to move the bot using the trained collision model.

## Load the trained model

An h5 file can be used or a folder path can be provided that contains the saved TensorFlow model.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import load_img, img_to_array

model = keras.models.load_model('tf_collision_model.h5')

### Load required modules

Modules required are for camera, as well converting raw frames into input Tensors for the model. The input shape should be (224, 224, 3). I.e. 3 colour channels. Traitlet widgets allows visualization of camera. Jetbot module controls the Adafruit motor drivers via PWM.

In [ ]:
import cv2
import numpy as np

import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg, Robot

### Instantiate Widgets and Robot

In [ ]:
camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)
blocked_slider = widgets.FloatSlider(description='blocked', min=0.0, max=1.0, orientation='vertical')
speed_slider = widgets.FloatSlider(description='speed', min=0.0, max=0.5, value=0.0, step=0.01, orientation='horizontal')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

robot = Robot()

display(widgets.VBox([widgets.HBox([image, blocked_slider]), speed_slider]))

Next, we'll create a function that will get called whenever the camera's value changes.  This function will do the following steps

1. Pre-process the camera image
2. Execute the neural network
3. While the neural network output indicates we're blocked, we'll turn left, otherwise we go forward.

In [ ]:
import time

image_size = 224

def update(change):
    global blocked_slider, robot
    x = change['new'] 
    img = load_img(img_path, color_mode='rgb', target_size=(image_size, image_size))
    img_arr = img_to_array(img=img)
    img_arr = np.array([img_arr]) * 1.0 / 255
    predict = model.predict(img_arr)
    
    blocked_slider.value = 1-predict
    
    if predict < 0.75:
        robot.forward(speed_slider.value)
    else:
        robot.left(speed_slider.value)
    
    time.sleep(0.001)
        
update({'new': camera.value}) 

Attach the update function to the value of the camera. This feeds the live image to the update function whereby the robot makes a decision as to whether to move forward in a free path or turn to avoid an obstacle.

In [ ]:
camera.observe(update, names='value')

### Emergency Stop

In [ ]:
import time

camera.unobserve(update, names='value')

time.sleep(0.1)  # add a small sleep to make sure frames have finished processing

robot.stop()

Run without streaming video to the browser. Camera will unlink.

In [ ]:
camera_link.unlink()  # don't stream to browser (will still run camera)

Continue streaming...

In [ ]:
camera_link.link()  # stream to browser (wont run camera)

Close the camera conneciton when complete.

In [ ]:
camera.stop()